<font color ='Orange'> <font size="9"> Challenge Dados Alura </font>

![title](../imagens/logo.png)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
import random
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn import set_config
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedKFold
from IPython.display import Image

## Carregando os dados

In [2]:
dados = pd.read_csv("../dados/DadosFinalPt1.csv", index_col = 0)

In [3]:
dados.head()

,CustomerID,Churn,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Monthly,Total
0,0002-ORFBO,0,Female,No,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,0,Male,No,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,1,Male,No,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,1,Male,Yes,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,1,Female,Yes,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        7043 non-null   object 
 1   Churn             7043 non-null   int64  
 2   Gender            7043 non-null   object 
 3   SeniorCitizen     7043 non-null   object 
 4   Partner           7043 non-null   object 
 5   Dependents        7043 non-null   object 
 6   Tenure            7043 non-null   int64  
 7   PhoneService      7043 non-null   object 
 8   MultipleLines     7043 non-null   object 
 9   InternetService   7043 non-null   object 
 10  OnlineSecurity    7043 non-null   object 
 11  OnlineBackup      7043 non-null   object 
 12  DeviceProtection  7043 non-null   object 
 13  TechSupport       7043 non-null   object 
 14  StreamingTV       7043 non-null   object 
 15  StreamingMovies   7043 non-null   object 
 16  Contract          7043 non-null   object 


In [5]:
# Irei excluir a variável Total porque ela é a junção de meses de contrato com o valor pago mensal. Isso foi realizado para que o modelo não fique enviesado
dados2 = dados.drop(['CustomerID', 'Total'], axis = 1).copy()

In [6]:
dados2.head()

,Churn,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Monthly
0,0,Female,No,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6
1,0,Male,No,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9
2,1,Male,No,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9
3,1,Male,Yes,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0
4,1,Female,Yes,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9


## Verificando valores únicos

In [7]:
def verificar_valores_unicos(tab, lista_colunas):
    for col in lista_colunas:
        result = tab[col].value_counts()
        print(f"Total de Registros em {col}")
        print(result)
        print("\n")

In [8]:
verificar_valores_unicos(dados2, dados2.columns)

Total de Registros em Churn
0    5174
1    1869
Name: Churn, dtype: int64


Total de Registros em Gender
Male      3555
Female    3488
Name: Gender, dtype: int64


Total de Registros em SeniorCitizen
No     5901
Yes    1142
Name: SeniorCitizen, dtype: int64


Total de Registros em Partner
No     3641
Yes    3402
Name: Partner, dtype: int64


Total de Registros em Dependents
No     4933
Yes    2110
Name: Dependents, dtype: int64


Total de Registros em Tenure
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: Tenure, Length: 73, dtype: int64


Total de Registros em PhoneService
Yes    6361
No      682
Name: PhoneService, dtype: int64


Total de Registros em MultipleLines
No     4072
Yes    2971
Name: MultipleLines, dtype: int64


Total de Registros em InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64


Total de Registros em OnlineSecurity
No     5024
Yes    2019

## Separando em categóricas e numéricas

In [9]:
var_cat = [colname for colname, coltype in dados2.dtypes.items() if dados2[colname].dtype == object]

In [10]:
var_cat

['Gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [11]:
var_num = [colnames for colnames, coltype in dados2.dtypes.items() if dados2[colnames].dtype != object and colnames != 'Churn']

In [12]:
var_num

['Tenure', 'Monthly']

In [13]:
target = dados2.columns[0]

In [14]:
verificar_valores_unicos(dados2, dados2[var_cat])

Total de Registros em Gender
Male      3555
Female    3488
Name: Gender, dtype: int64


Total de Registros em SeniorCitizen
No     5901
Yes    1142
Name: SeniorCitizen, dtype: int64


Total de Registros em Partner
No     3641
Yes    3402
Name: Partner, dtype: int64


Total de Registros em Dependents
No     4933
Yes    2110
Name: Dependents, dtype: int64


Total de Registros em PhoneService
Yes    6361
No      682
Name: PhoneService, dtype: int64


Total de Registros em MultipleLines
No     4072
Yes    2971
Name: MultipleLines, dtype: int64


Total de Registros em InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64


Total de Registros em OnlineSecurity
No     5024
Yes    2019
Name: OnlineSecurity, dtype: int64


Total de Registros em OnlineBackup
No     4614
Yes    2429
Name: OnlineBackup, dtype: int64


Total de Registros em DeviceProtection
No     4621
Yes    2422
Name: DeviceProtection, dtype: int64


Total de Registros em 

In [15]:
dados2[var_num].describe()

,Tenure,Monthly
count,7043.000000,7043.000000
mean,32.371149,64.761692
std,24.559481,30.090047
min,0.000000,18.250000
25%,9.000000,35.500000
50%,29.000000,70.350000
75%,55.000000,89.850000
max,72.000000,118.750000


## Binarização

Para as variáveis que possuem apenas 2 categorias (Não ou Sim), irei transformá-las em 0 ou 1

In [16]:
# Aqui filtra variáveis categóricas com 2 categorias (nunique menor que 3)
binarizar = dados2[var_cat].loc[:,dados2.nunique()<3].columns

In [17]:
binarizar

Index(['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling'],
      dtype='object')

In [18]:
# Tirar o Gender porque as categorias são diferentes
binarizar = binarizar.drop('Gender')

In [19]:
# dicionário para posteriomente transformar Female em 0 e Male em 1
gender_dict = {'Female':0, 'Male':1}

In [20]:
# usando o replace para transformar Female em 0 e Male em 1
dados2['Gender'] = dados2['Gender'].replace(gender_dict)

In [21]:
# verificando as colunas que serão alteradas
dados2[binarizar].head()

,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling
0,No,Yes,Yes,Yes,No,No,Yes,No,Yes,Yes,No,Yes
1,No,No,No,Yes,Yes,No,No,No,No,No,Yes,No
2,No,No,No,Yes,No,No,No,Yes,No,No,No,Yes
3,Yes,Yes,No,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes
4,Yes,Yes,No,Yes,No,No,No,No,Yes,Yes,No,Yes


In [22]:
# usando np.where para transformar "No" em 0 e "Yes" em 1
dados2[binarizar] = np.where(dados2[binarizar] == 'No', 0, 1)

## One Hot Encoder e Standard Scaler

In [23]:
from imblearn.pipeline import Pipeline as ImblearnPipeline
from sklearn.compose import make_column_transformer

<b> One Hot Encoder</b> é uma técnica para transformar os dados categóricos em numéricos, possibilitando que seja lido pelos algoritmos de Machine Learning.
Ele consistem em transformar as categorias de cada variável em coluna e preenchendo com 0 ou 1.

<b>Exemplo:</b> <br> Para a primeira linha o cliente possui Internet Service do tipo DSL <BR>
Enquanto a segunda linha é Fiber Optic <br>
A terceira linha Não possui InternetService    

![title](../imagens/OneHotEncoder.png)

In [24]:
# selecionando as variáveis que possuem mais de 2 categorias para fazer o One Hot Encoder
cat_one_hot = list(dados2[var_cat].loc[:,dados2.nunique()>=3].columns)
cat_one_hot

['InternetService', 'Contract', 'PaymentMethod']

Aqui irei realizar a padronização dos dados numéricos, porque alguns algoritmos de Machine Learning podem ficar tendenciosos para alguns valores, porque por exemplo, imagine que temos um valor 5 e outro 500, o algoritmo pode tender para este 500 (bias).
Por isso aplicamos técnicas de padronização, no caso, aplicarei o método Standard()

A Regressão Logística e o SVC são sensíveis a dados não padronizados, por isso decidi deixar tudo padronizado em todos os algoritmos

Posteriormente irei usar o mesmo pipeline para aplicar nos dados de treino e dados novos.

Perceba na tabela dados_processados que os dados agora se encontram em uma mesma escala

In [25]:
# Essas variáveis serão padronizadas
var_num

['Tenure', 'Monthly']

Aqui está sendo configurado o Pipeline que permite realizar mais de uma transformação ao mesmo tempo.
Serão realizados o One Hot Encoder e Padronização (Standard)

In [26]:
set_config(transform_output="pandas")
numeric_features = ["Tenure","Monthly"]
numeric_transformer = Pipeline(
        steps=[("scaler", StandardScaler())]
    )

categorical_features = ['InternetService', 'Contract', 'PaymentMethod']
categorical_transformer = Pipeline(
        steps=[
            ("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore", dtype = np.int64))
            ]
    )
preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ], remainder= 'passthrough'
    )

pipeline = Pipeline(
    steps=[("preprocessor", preprocessor)]
    )

dados_processados = pipeline.fit_transform(dados2)

In [27]:
dados_processados

,num__Tenure,num__Monthly,cat__InternetService_DSL,cat__InternetService_Fiber optic,cat__InternetService_No,cat__Contract_Month-to-month,cat__Contract_One year,cat__Contract_Two year,cat__PaymentMethod_Bank transfer (automatic),cat__PaymentMethod_Credit card (automatic),...,remainder__Dependents,remainder__PhoneService,remainder__MultipleLines,remainder__OnlineSecurity,remainder__OnlineBackup,remainder__DeviceProtection,remainder__TechSupport,remainder__StreamingTV,remainder__StreamingMovies,remainder__PaperlessBilling
0,-0.951682,0.027862,1,0,0,0,1,0,0,0,...,1,1,0,0,1,0,1,1,0,1
1,-0.951682,-0.161583,1,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
2,-1.155283,0.303720,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,-0.788800,1.104706,0,1,0,1,0,0,0,0,...,0,1,0,0,1,1,0,1,1,1
4,-1.196004,0.636080,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,-0.788800,-0.319454,1,0,0,0,1,0,0,0,...,0,1,0,1,0,0,1,0,0,0
7263,-0.422317,0.675963,0,1,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,1,1
7264,-1.236724,-0.480648,1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
7265,1.410099,0.102643,1,0,0,0,0,1,0,0,...,1,1,0,1,0,1,1,0,1,0


In [28]:
dados_processados = dados_processados.drop('remainder__Churn', axis = 1)

In [29]:
X = dados_processados

In [30]:
X.columns

Index(['num__Tenure', 'num__Monthly', 'cat__InternetService_DSL',
       'cat__InternetService_Fiber optic', 'cat__InternetService_No',
       'cat__Contract_Month-to-month', 'cat__Contract_One year',
       'cat__Contract_Two year',
       'cat__PaymentMethod_Bank transfer (automatic)',
       'cat__PaymentMethod_Credit card (automatic)',
       'cat__PaymentMethod_Electronic check',
       'cat__PaymentMethod_Mailed check', 'remainder__Gender',
       'remainder__SeniorCitizen', 'remainder__Partner',
       'remainder__Dependents', 'remainder__PhoneService',
       'remainder__MultipleLines', 'remainder__OnlineSecurity',
       'remainder__OnlineBackup', 'remainder__DeviceProtection',
       'remainder__TechSupport', 'remainder__StreamingTV',
       'remainder__StreamingMovies', 'remainder__PaperlessBilling'],
      dtype='object')

In [31]:
y = dados2[target]


## Aplicando SPLIT e SMOTE

Realizando o split dos dados, para separar em treino e teste.
Os dados de treino passarão pela técnica SMOTE que é uma técnica para criar dados sintéticos para a categoria que está em menor número, no caso, temos menos dados para clientes que realizaram o cancelamento da assinatura (Churn = Yes/ 1)

In [32]:
# Aplicando o Split dos dados. 80% serão dedicados para treino do modelo e 20% para treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Aplicar o SMOTE no conjunto de treinamento para lidar com o desequilíbrio de classes
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [33]:
X_train_resampled

,num__Tenure,num__Monthly,cat__InternetService_DSL,cat__InternetService_Fiber optic,cat__InternetService_No,cat__Contract_Month-to-month,cat__Contract_One year,cat__Contract_Two year,cat__PaymentMethod_Bank transfer (automatic),cat__PaymentMethod_Credit card (automatic),...,remainder__Dependents,remainder__PhoneService,remainder__MultipleLines,remainder__OnlineSecurity,remainder__OnlineBackup,remainder__DeviceProtection,remainder__TechSupport,remainder__StreamingTV,remainder__StreamingMovies,remainder__PaperlessBilling
0,1.125057,0.975086,0,1,0,1,0,0,0,1,...,0,1,1,1,1,0,0,0,1,1
1,-0.829521,-1.509300,0,0,1,0,0,1,1,0,...,1,1,0,0,0,0,0,0,0,0
2,-1.236724,0.521416,0,1,0,1,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
3,0.025607,0.331971,1,0,0,0,1,0,0,1,...,1,1,0,1,0,1,0,1,1,0
4,-0.748080,0.705875,0,1,0,1,0,0,0,0,...,1,1,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8273,-0.458209,0.522386,0,1,0,1,0,0,0,0,...,0,1,1,0,1,0,0,0,0,1
8274,1.556937,1.385693,0,1,0,0,0,1,0,0,...,0,1,1,1,0,0,1,1,1,1
8275,-0.717878,-0.634121,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
8276,0.888064,1.494780,0,1,0,0,0,1,1,0,...,0,1,0,1,1,1,0,1,1,0


## Stratified k-fold

Importante empregar técnicas de validação cruzada para avaliar a capacidade prática do modelo.

Esta técnica consiste em dividir os dados em k partições, e utilizar uma partição para teste e as demais para estimar os hiperparâmetros, essa técnica irá treinar "diferentes modelos" para cada partição.

Irá garantir que cada partição tenha os dados proporcionais, ou seja, churn 0 e 1 terão a mesma proporção para que não haja problemas do algoritmo aprender "de mais" a classe majoritária.

Estamos buscando um algoritmo que saiba identificar nuances, se o algortimo aprende de mais alguma classe, ele não será generalizável, estará enviesado. Além disso, está sendo treinado com shuffle = True, garante a aleatoriedade.


In [34]:
# Inicializar o StratifiedKFold para validação cruzada
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

## Construção dos Modelos de Machine Learning

Primeiramente inicializei o modelo, depois criei uma grade com os parâmetros que serão testados

informei no objeto GridSearch e apliquei o "fit" para os dados de treino.

Depois capturei os melhores parâmetros e treinei o modelo com estes melhores.

Realizei a previsão usando o "predict" e "predict_proba" (que gera a probabilidade)

Então, salvei as métricas do modelo, acurácia, precisão, recall (sensibilidade) e F1.
Por fim, nos modelos que possuem a opção de "feature importance", printei as variáveis pela ordem de importância na criação do modelo.

<b> Acurácia</b> <br>
Mede o quão exato foi o modelo para classificar.

Mostra a proporção de previsões corretas pelo total de registros.  (tanto os verdadeiros positivos quanto os verdadeiros negativos) 

![title](../imagens/acuracia.png)

<b>Precisão</b> <br>
Mede a proporção entre casos positivos com todos os casos positivos. Clientes identificados corretamente como Churn

![title](../imagens/precisao.png)

<b>Recall</b> <br>

Mede a sensibilidade do modelo, o quão bem o modelo consegue prever o churn. Quanto mais próximo de 100, melhor, porque indica que o modelo não está gerando muitos falsos negativos, ou seja, o cliente tem probabilidade de sair, porém é identificado como não.

![title](../imagens/recall.png)

<b>F1-Score</b> <br>

é média harmônica entre a precisão e o recall, esta métrica será escolhida porque queremos uma boa precisão e um bom recall.

![title](../imagens/f1score.png)

<b> AUC </b> <br>

Esta métrica é útil para comparação de modelos diferentes, mede a área sob a curva

### Modelo 1  (Benchmark) -  Regressão Logística

O modelo Benchmark é aquele mais simples, para servir de comparação com os demais, para que possamos analisar se os modelos estão tendo um melhor desempenho ou não.

In [35]:
# Inicializar o modelo e a grade de parâmetros

model_v1 = LogisticRegression()

param_grid_v1 = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 
                   'penalty': ['l1', 'l2'],
                   'solver': ['liblinear']}

# Inicializar o GridSearchCV com o modelo, a grade de parâmetros e a métrica de pontuação
grid_search_v1 = GridSearchCV(
    estimator=model_v1,
    param_grid=param_grid_v1,
    scoring='f1',  
    cv=skf,  
    verbose=1,
    n_jobs = -1
    
)

# Realizar a busca em grade usando os dados ressampled
grid_search_v1.fit(X_train_resampled, y_train_resampled)

# Imprimir os melhores parâmetros e a pontuação F1 correspondente
print("Best parameters found: ", grid_search_v1.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search_v1.best_score_))

# Salvar o melhor F1 para comparar com o teste
best_score_v1 = grid_search_v1.best_score_

# Treinar o modelo final usando os melhores hiperparâmetros
best_params_v1 = grid_search_v1.best_params_
final_model_v1 = LogisticRegression(**best_params_v1)
final_model_v1.fit(X_train_resampled, y_train_resampled)

# # Previsões com dados de teste
y_pred_v1 = final_model_v1.predict(X_test)


# # Obtemos as previsões no formato de probabilidade filtrando para a classe positiva
pred_prob_v1 = final_model_v1.predict_proba(X_test)[:,1]


# Métricas
accuracy_v1 = accuracy_score(y_test, y_pred_v1)
precision_v1 = precision_score(y_test, y_pred_v1)
recall_v1 = recall_score(y_test, y_pred_v1)
f1_v1 = f1_score(y_test, y_pred_v1)


# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v1 = roc_auc_score(y_test, y_pred_v1)

# Calcula a curva ROC com dados e previsões em teste
fpr_v1, tpr_v1, thresholds = roc_curve(y_test, pred_prob_v1)


# AUC em teste
auc_v1 = auc(fpr_v1, tpr_v1)


# Extrair features importances

# Obtemos os coeficientes pelo maior maior usando np.argsort
indices = np.argsort(-abs(final_model_v1.coef_[0,:]))


print("Variáveis mais importantes para o resultado do modelo_v1:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)  


Fitting 7 folds for each of 18 candidates, totalling 126 fits
Best parameters found:  {'C': 10000, 'penalty': 'l2', 'solver': 'liblinear'}
Best F1 score found: 0.82
Variáveis mais importantes para o resultado do modelo_v1:
--------------------------------------------------
cat__InternetService_Fiber optic
num__Monthly
cat__Contract_Two year
cat__PaymentMethod_Credit card (automatic)
cat__PaymentMethod_Bank transfer (automatic)
cat__PaymentMethod_Mailed check
cat__Contract_One year
cat__PaymentMethod_Electronic check
remainder__PhoneService
cat__Contract_Month-to-month
cat__InternetService_DSL
remainder__StreamingMovies
remainder__StreamingTV
remainder__TechSupport
remainder__OnlineSecurity
remainder__OnlineBackup
remainder__DeviceProtection
cat__InternetService_No
remainder__MultipleLines
num__Tenure
remainder__Dependents
remainder__PaperlessBilling
remainder__Gender
remainder__Partner
remainder__SeniorCitizen


In [36]:
# Salva o modelo em disco
with open('../modelos/modelo_v1.pkl', 'wb') as pickle_file:
    pickle.dump(final_model_v1, pickle_file)

In [37]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v1 = {'Nome': 'modelo_v1', 
                  'Algoritmo': 'Regressão Logística', 
                  'ROC_AUC Score': roc_auc_v1,
                  'AUC Score': auc_v1,
                  'Acurácia': accuracy_v1,
                  'Precisão': precision_v1,
                  'Recall': recall_v1,
                  'F1 Score': f1_v1,
                  'Best F1 Treino': best_score_v1,
                 }

In [38]:
# Cria um dataframe para receber as métricas de cada modelo
df_modelos = pd.DataFrame(dict_modelo_v1, index = [0])

### Modelo 2 - Random Forest

In [39]:
# Inicializar o modelo e a grade de parâmetros

model_v2 = RandomForestClassifier()

param_grid_v2 = {
    'n_estimators': range(60, 120, 20),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 8, 2),
    'min_samples_split': range(2, 10, 2),
    'min_samples_leaf': [2, 3],
    'bootstrap': [True, False]
   
}


# Inicializar o GridSearchCV com o modelo, a grade de parâmetros e a métrica de pontuação
grid_search_v2 = GridSearchCV(
    estimator=model_v2,
    param_grid=param_grid_v2,
    scoring='f1',  # Use a métrica de pontuação apropriada para o seu problema
    cv=skf,  # StratifiedKFold como estratégia de validação cruzada
    verbose=1  # Controla a verbosidade; ajuste conforme necessário
)

# Realizar a busca em grade usando os dados ressampled
grid_search_v2.fit(X_train_resampled, y_train_resampled)

# Imprimir os melhores parâmetros e a pontuação F1 correspondente
print("Best parameters found: ", grid_search_v2.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search_v2.best_score_))

# Salvar o melhor F1 para comparar com o teste
best_score_v2 = grid_search_v2.best_score_

# Treinar o modelo final usando os melhores hiperparâmetros
best_params_v2 = grid_search_v2.best_params_
final_model_v2 = RandomForestClassifier(**best_params_v2)
final_model_v2.fit(X_train_resampled, y_train_resampled)

# # Previsões com dados de teste
y_pred_v2 = final_model_v2.predict(X_test)


# # Obtemos as previsões no formato de probabilidade filtrando para a classe positiva
pred_prob_v2 = final_model_v2.predict_proba(X_test)[:,1]


# Métricas
accuracy_v2 = accuracy_score(y_test, y_pred_v2)
precision_v2 = precision_score(y_test, y_pred_v2)
recall_v2 = recall_score(y_test, y_pred_v2)
f1_v2 = f1_score(y_test, y_pred_v2)


# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v2 = roc_auc_score(y_test, y_pred_v2)

# Calcula a curva ROC com dados e previsões em teste
fpr_v2, tpr_v2, thresholds2 = roc_curve(y_test, pred_prob_v2)


# AUC em teste
auc_v2 = auc(fpr_v2, tpr_v2)


# Extrair features importances

# Variáveis mais relevantes
indices = np.argsort(-final_model_v2.feature_importances_)
print("Variáveis mais importantes para o resultado do modelo_v2:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)

Fitting 7 folds for each of 288 candidates, totalling 2016 fits
Best parameters found:  {'bootstrap': True, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 60}
Best F1 score found: 0.80
Variáveis mais importantes para o resultado do modelo_v2:
--------------------------------------------------
cat__Contract_Month-to-month
num__Tenure
cat__Contract_Two year
cat__InternetService_Fiber optic
cat__InternetService_No
num__Monthly
remainder__TechSupport
cat__PaymentMethod_Electronic check
remainder__OnlineSecurity
cat__Contract_One year
remainder__Dependents
cat__InternetService_DSL
remainder__OnlineBackup
cat__PaymentMethod_Credit card (automatic)
remainder__Partner
remainder__StreamingMovies
cat__PaymentMethod_Bank transfer (automatic)
remainder__DeviceProtection
remainder__Gender
remainder__PaperlessBilling
cat__PaymentMethod_Mailed check
remainder__MultipleLines
remainder__PhoneService
remainder__StreamingTV
remainder__SeniorCitizen


In [40]:
# Salva o modelo em disco
with open('../modelos/modelo_v2.pkl', 'wb') as pickle_file:
    pickle.dump(final_model_v2, pickle_file)

In [41]:
new_row = pd.Series({'Nome': 'modelo_v2', 
                  'Algoritmo': 'Random Forest', 
                  'ROC_AUC Score': roc_auc_v2,
                  'AUC Score': auc_v2,
                  'Acurácia': accuracy_v2,
                  'Precisão': precision_v2,
                  'Recall': recall_v2,   
                  'F1 Score': f1_v2,
                  'Best F1 Treino': best_score_v2,
                  })

In [42]:
df_modelos = pd.concat([df_modelos, new_row.to_frame().T], ignore_index=True)

### Modelo 3 - SVM Support Vector Machine - Classifier

In [43]:
# Initialize the model and parameter grid
model_v3 = SVC(kernel = 'rbf')

param_grid_v3 = {    
    
    'C': [0.001, 0.01, 0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1],
   
}



# Initialize GridSearchCV with the model, parameter grid, and scoring metric
grid_search_v3 = GridSearchCV(
    estimator=model_v3,
    param_grid=param_grid_v3,
    scoring='f1',  # Use appropriate scoring metric for your problem
    cv=skf,  # StratifiedKFold as the cross-validation strategy
    verbose=1  # Controls the verbosity; adjust as needed
)

# Realizar a busca em grade usando os dados ressampled
grid_search_v3.fit(X_train_resampled, y_train_resampled)

# Imprimir os melhores parâmetros e a pontuação F1 correspondente
print("Best parameters found: ", grid_search_v3.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search_v3.best_score_))

# Salvar o melhor F1 para comparar com o teste
best_score_v3 = grid_search_v3.best_score_

# Train the final model using the best hyperparameters
best_params_v3 = grid_search_v3.best_params_
final_model_v3 = SVC(**best_params_v3, probability=True)
final_model_v3.fit(X_train_resampled, y_train_resampled)


# # Previsões com dados de teste
y_pred_v3 = final_model_v3.predict(X_test)


# # Obtemos as previsões no formato de probabilidade filtrando para a classe positiva
pred_prob_v3 = final_model_v3.predict_proba(X_test)[:,1]


# Métricas
accuracy_v3 = accuracy_score(y_test, y_pred_v3)
precision_v3 = precision_score(y_test, y_pred_v3)
recall_v3 = recall_score(y_test, y_pred_v3)
f1_v3 = f1_score(y_test, y_pred_v3)


# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v3 = roc_auc_score(y_test, y_pred_v3)

# Calcula a curva ROC com dados e previsões em teste
fpr_v3, tpr_v3, thresholds3 = roc_curve(y_test, pred_prob_v3)


# AUC em teste
auc_v3 = auc(fpr_v3, tpr_v3)

Fitting 7 folds for each of 20 candidates, totalling 140 fits
Best parameters found:  {'C': 10, 'gamma': 1}
Best F1 score found: 0.84


In [44]:
# Salva o modelo em disco
with open('../modelos/modelo_v3.pkl', 'wb') as pickle_file:
    pickle.dump(final_model_v3, pickle_file)

In [45]:
# Dicionário com as métricas do modelo_v3
new_row = pd.Series({'Nome': 'modelo_v3', 
                  'Algoritmo': 'SVC', 
                  'ROC_AUC Score': roc_auc_v3,
                  'AUC Score': auc_v3,
                  'Acurácia': accuracy_v3,
                  'Precisão': precision_v3,
                  'Recall': recall_v3,
                  'F1 Score': f1_v3,
                  'Best F1 Treino': best_score_v3})

In [46]:
df_modelos = pd.concat([df_modelos, new_row.to_frame().T], ignore_index=True)

### Modelo 4 - XGBoost

In [47]:
import xgboost as xgb

In [48]:
# Initialize the model and parameter grid
model_v4 = xgb.XGBClassifier(booster='gbtree', random_state=42)

param_grid_v4 = {    
    
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001]
   
}



# Initialize GridSearchCV with the model, parameter grid, and scoring metric
grid_search_v4 = GridSearchCV(
    estimator=model_v4,
    param_grid=param_grid_v4,
    scoring='f1',  # Use appropriate scoring metric for your problem
    cv=skf,  # StratifiedKFold as the cross-validation strategy
    verbose=1  # Controls the verbosity; adjust as needed
)

# Realizar a busca em grade usando os dados ressampled
grid_search_v4.fit(X_train_resampled, y_train_resampled)

# Imprimir os melhores parâmetros e a pontuação F1 correspondente
print("Best parameters found: ", grid_search_v4.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search_v4.best_score_))

# Salvar o melhor F1 para comparar com o teste
best_score_v4 = grid_search_v4.best_score_

# Train the final model using the best hyperparameters
best_params_v4 = grid_search_v4.best_params_
final_model_v4 = xgb.XGBClassifier(**best_params_v4, booster='gbtree', random_state=42)
final_model_v4.fit(X_train_resampled, y_train_resampled)


# # Previsões com dados de teste
y_pred_v4 = final_model_v4.predict(X_test)


# # Obtemos as previsões no formato de probabilidade filtrando para a classe positiva
pred_prob_v4 = final_model_v4.predict_proba(X_test)[:,1]


# Métricas
accuracy_v4 = accuracy_score(y_test, y_pred_v4)
precision_v4 = precision_score(y_test, y_pred_v4)
recall_v4 = recall_score(y_test, y_pred_v4)
f1_v4 = f1_score(y_test, y_pred_v4)


# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v4 = roc_auc_score(y_test, y_pred_v4)

# Calcula a curva ROC com dados e previsões em teste
fpr_v4, tpr_v4, thresholds4 = roc_curve(y_test, pred_prob_v4)


# AUC em teste
auc_v4 = auc(fpr_v4, tpr_v4)


# Extrair features importances
importance_type = 'weight'

feature_importance = final_model_v4.get_booster().get_score(importance_type=importance_type)

# Variáveis mais relevantes
indices = np.argsort(-final_model_v4.feature_importances_)
print("Variáveis mais importantes para o resultado do modelo_v2:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)


Fitting 7 folds for each of 27 candidates, totalling 189 fits
Best parameters found:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
Best F1 score found: 0.84
Variáveis mais importantes para o resultado do modelo_v2:
--------------------------------------------------
cat__Contract_Month-to-month
cat__InternetService_No
cat__InternetService_Fiber optic
cat__Contract_Two year
cat__Contract_One year
remainder__TechSupport
cat__PaymentMethod_Electronic check
remainder__OnlineSecurity
remainder__Dependents
remainder__PhoneService
num__Tenure
cat__PaymentMethod_Credit card (automatic)
cat__InternetService_DSL
remainder__StreamingMovies
remainder__OnlineBackup
cat__PaymentMethod_Bank transfer (automatic)
remainder__Gender
cat__PaymentMethod_Mailed check
num__Monthly
remainder__MultipleLines
remainder__PaperlessBilling
remainder__DeviceProtection
remainder__SeniorCitizen
remainder__StreamingTV
remainder__Partner


In [49]:
# Salva o modelo em disco
with open('../modelos/modelo_v4.pkl', 'wb') as pickle_file:
    pickle.dump(final_model_v4, pickle_file)

In [50]:
# Dicionário com as métricas do modelo_v4
new_row = pd.Series({'Nome': 'modelo_v4', 
                  'Algoritmo': 'XGBoost', 
                  'ROC_AUC Score': roc_auc_v4,
                  'AUC Score': auc_v4,
                  'Acurácia': accuracy_v4,
                  'Precisão': precision_v4,
                  'Recall': recall_v4,
                  'F1 Score': f1_v4,
                  'Best F1 Treino': best_score_v4})

In [51]:
df_modelos = pd.concat([df_modelos, new_row.to_frame().T], ignore_index=True)

In [52]:
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia,Precisão,Recall,F1 Score,Best F1 Treino
0,modelo_v1,Regressão Logística,0.744212,0.826857,0.767211,0.548523,0.695187,0.613208,0.820579
1,modelo_v2,Random Forest,0.759973,0.841267,0.745209,0.512998,0.791444,0.622503,0.804844
2,modelo_v3,SVC,0.696208,0.760221,0.759404,0.545455,0.561497,0.55336,0.843397
3,modelo_v4,XGBoost,0.728248,0.814539,0.765082,0.548533,0.649733,0.594859,0.839282


## Testar tirando os parâmetros Gender e PhoneService

### Modelo 5 - Random Forest  v2

In [53]:
#from imblearn.under_sampling import RandomUnderSampler
#from sklearn.metrics import confusion_matrix

In [54]:
dados_processados_v2 = dados_processados.drop(['remainder__Gender','remainder__PhoneService'], axis = 1)

In [55]:
X_v3 = dados_processados_v2

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_v3, y, test_size=0.2, random_state=42, stratify=y)

# Aplicar o SMOTE no conjunto de treinamento para lidar com o desequilíbrio de classes
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [57]:
X_train_resampled

,num__Tenure,num__Monthly,cat__InternetService_DSL,cat__InternetService_Fiber optic,cat__InternetService_No,cat__Contract_Month-to-month,cat__Contract_One year,cat__Contract_Two year,cat__PaymentMethod_Bank transfer (automatic),cat__PaymentMethod_Credit card (automatic),...,remainder__Partner,remainder__Dependents,remainder__MultipleLines,remainder__OnlineSecurity,remainder__OnlineBackup,remainder__DeviceProtection,remainder__TechSupport,remainder__StreamingTV,remainder__StreamingMovies,remainder__PaperlessBilling
0,1.125057,0.975086,0,1,0,1,0,0,0,1,...,1,0,1,1,1,0,0,0,1,1
1,-0.829521,-1.509300,0,0,1,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,-1.236724,0.521416,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
3,0.025607,0.331971,1,0,0,0,1,0,0,1,...,1,1,0,1,0,1,0,1,1,0
4,-0.748080,0.705875,0,1,0,1,0,0,0,0,...,0,1,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8273,-0.239544,0.504321,0,1,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
8274,1.556937,1.385693,0,1,0,0,0,1,0,0,...,0,0,1,1,0,0,1,1,1,1
8275,-1.236724,-0.659530,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8276,0.910051,1.494855,0,1,0,0,0,0,1,0,...,1,0,0,1,1,1,1,1,1,0


In [58]:
# Inicializar o modelo e a grade de parâmetros

model_v5 = RandomForestClassifier()

param_grid_v5 = {'n_estimators': range(60, 120, 20),
                 'criterion': ['gini', 'entropy'],
                 'max_depth': range(2, 8, 2),
                 'min_samples_split': range(2, 10, 2),
                 'min_samples_leaf': [2, 3],
                 'bootstrap': [True, False]
                }

# Inicializar o GridSearchCV com o modelo, a grade de parâmetros e a métrica de pontuação
grid_search_v5 = GridSearchCV(
    estimator=model_v5,
    param_grid=param_grid_v5,
    scoring='f1',  # Use a métrica de pontuação apropriada para o seu problema
    cv=skf,  # StratifiedKFold como estratégia de validação cruzada
    verbose=1  # Controla a verbosidade; ajuste conforme necessário
)

# Realizar a busca em grade usando os dados ressampled
grid_search_v5.fit(X_train_resampled, y_train_resampled)

# Imprimir os melhores parâmetros e a pontuação F1 correspondente
print("Best parameters found: ", grid_search_v5.best_params_)
print("Best F1 score found: {:.2f}".format(grid_search_v5.best_score_))

# Salvar o melhor F1 para comparar com o teste
best_score_v5 = grid_search_v5.best_score_

# Treinar o modelo final usando os melhores hiperparâmetros
best_params_v5 = grid_search_v5.best_params_
final_model_v5 = RandomForestClassifier(**best_params_v5)
final_model_v5.fit(X_train_resampled, y_train_resampled)

# # Previsões com dados de teste
y_pred_v5 = final_model_v5.predict(X_test)


# # Obtemos as previsões no formato de probabilidade filtrando para a classe positiva
pred_prob_v5 = final_model_v5.predict_proba(X_test)[:,1]


# Métricas
accuracy_v5 = accuracy_score(y_test, y_pred_v5)
precision_v5 = precision_score(y_test, y_pred_v5)
recall_v5 = recall_score(y_test, y_pred_v5)
f1_v5 = f1_score(y_test, y_pred_v5)


# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v5 = roc_auc_score(y_test, y_pred_v5)

# Calcula a curva ROC com dados e previsões em teste
fpr_v5, tpr_v5, thresholds5 = roc_curve(y_test, pred_prob_v5)


# AUC em teste
auc_v5 = auc(fpr_v5, tpr_v5)


# Extrair features importances

# Variáveis mais relevantes
indices = np.argsort(-final_model_v5.feature_importances_)
print("Variáveis mais importantes para o resultado do modelo_v2:")
print(50*'-')
for feature in X_v3.columns[indices]:
    print(feature)
    


Fitting 7 folds for each of 288 candidates, totalling 2016 fits
Best parameters found:  {'bootstrap': True, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 100}
Best F1 score found: 0.80
Variáveis mais importantes para o resultado do modelo_v2:
--------------------------------------------------
cat__Contract_Month-to-month
num__Tenure
cat__Contract_Two year
cat__InternetService_Fiber optic
num__Monthly
cat__Contract_One year
cat__InternetService_No
remainder__TechSupport
cat__PaymentMethod_Electronic check
remainder__OnlineSecurity
remainder__Partner
remainder__Dependents
cat__InternetService_DSL
remainder__OnlineBackup
remainder__PaperlessBilling
cat__PaymentMethod_Bank transfer (automatic)
cat__PaymentMethod_Credit card (automatic)
remainder__StreamingMovies
remainder__StreamingTV
remainder__DeviceProtection
cat__PaymentMethod_Mailed check
remainder__MultipleLines
remainder__SeniorCitizen


In [59]:
# Salva o modelo em disco
with open('../modelos/modelo_v5.pkl', 'wb') as pickle_file:
    pickle.dump(final_model_v5, pickle_file)

In [60]:
# Dicionário com as métricas do modelo_v5
new_row = pd.Series({'Nome': 'modelo_v5', 
                  'Algoritmo': 'Random Forest Menos Features', 
                  'ROC_AUC Score': roc_auc_v5,
                  'AUC Score': auc_v5,
                  'Acurácia': accuracy_v5,
                  'Precisão': precision_v5,
                  'Recall': recall_v5,
                  'F1 Score': f1_v5,
                  'Best F1 Treino': best_score_v5})

In [61]:
df_modelos = pd.concat([df_modelos, new_row.to_frame().T], ignore_index=True)

In [62]:
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia,Precisão,Recall,F1 Score,Best F1 Treino
0,modelo_v1,Regressão Logística,0.744212,0.826857,0.767211,0.548523,0.695187,0.613208,0.820579
1,modelo_v2,Random Forest,0.759973,0.841267,0.745209,0.512998,0.791444,0.622503,0.804844
2,modelo_v3,SVC,0.696208,0.760221,0.759404,0.545455,0.561497,0.55336,0.843397
3,modelo_v4,XGBoost,0.728248,0.814539,0.765082,0.548533,0.649733,0.594859,0.839282
4,modelo_v5,Random Forest Menos Features,0.771635,0.842013,0.752307,0.521441,0.812834,0.635319,0.800691


O melhor modelo foi o Random Forest utilizando Menos Features, deu uma pequena melhora.